In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [11]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Mon Dec 12 10:08:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 40%   38C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/otto_ii/xgtrain.csv')
test_x = dd.read_csv('../input/otto_ii/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2022-12-12 10:08:54,517 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-12 10:08:54,518 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-12-12 10:08:54,527 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-12 10:08:54,527 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [12]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'num_class':9,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    lloss = log_loss(test_y, predictions)
    
    return lloss

In [13]:
study = optuna.create_study(direction='minimize')

[I 2022-12-12 10:13:37,696] A new study created in memory with name: no-name-ffe30005-b53b-446a-b6d4-5fd62db6a3d7


In [14]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:13:40] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0
[I 2022-12-12 10:13:51,346] Trial 0 finished with value: 0.6482739944996688 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 2.4406516077861133, 'alpha': 0.007277731535529787, 'colsample_bytree': 0.7137368683143084, 'subsample': 0.6925854864563181, 'learning_rate': 0.0664515743850402, 'max_depth': 6, 'min_child_weight': 253, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.6482739944996688.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:13:51] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0
[I 2022-12-12 10:14:10,938] Trial 1 finished with value: 1.065546345

CPU times: user 1.83 s, sys: 639 ms, total: 2.47 s
Wall time: 30.8 s


In [15]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:14:19] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0
[I 2022-12-12 10:18:38,868] Trial 2 finished with value: 0.7598243738289363 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 1.4617428110776984, 'alpha': 0.03162167812934222, 'colsample_bytree': 0.8848191364376565, 'subsample': 0.7104683009010917, 'learning_rate': 0.006890155889075234, 'max_depth': 21, 'min_child_weight': 270, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.6482739944996688.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:18:38] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0
[I 2022-12-12 10:19:02,811] Trial 3 finished with value: 0.5810898

CPU times: user 17min 29s, sys: 8min 45s, total: 26min 15s
Wall time: 11h 44min 42s


In [16]:
study.best_trial.params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.003535658536203966,
 'alpha': 0.3700613064637976,
 'colsample_bytree': 0.6605986441114581,
 'subsample': 0.6351546013190558,
 'learning_rate': 0.023875179448246714,
 'max_depth': 20,
 'min_child_weight': 7,
 'eval_metric': 'logloss'}

In [18]:
real_test_x = dd.read_csv('../input/otto_ii/xgtest.csv')

In [19]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [20]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [24]:
params = study.best_trial.params
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})


predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

[05:36:36] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


0.4460577599356624

In [25]:
%%time
params = study.best_trial.params
params['seed'] = 2022
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[05:40:10] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


CPU times: user 5.5 s, sys: 3.26 s, total: 8.76 s
Wall time: 3min 33s


0.44607520641023396

In [26]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['num_class']= 9
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    
    error = log_loss(real_test_y, predictions)

    scores.append(error)
    print(error)

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[05:43:44] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


0.44471311337813957
1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[05:47:18] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


0.44479705036477435
2


...

0.44500082056422163
13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[06:30:06] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


0.4451634947384916
14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[06:33:40] task [xgboost.dask-0]:tcp://127.0.0.1:40665 got new rank 0


0.4451258055867051
CPU times: user 1min 29s, sys: 39.1 s, total: 2min 8s
Wall time: 53min 29s


In [27]:
scores

[0.44471311337813957,
 0.44479705036477435,
 0.44513801684693344,
 0.44591324115388925,
 0.4463465332639558,
 0.4443313639509529,
 0.44606039330423625,
 0.4443926129003217,
 0.4454616336170991,
 0.44588380348101236,
 0.44572211710419885,
 0.44671325261880424,
 0.44500082056422163,
 0.4451634947384916,
 0.4451258055867051]

In [28]:
import numpy as np
np.mean(scores)

0.4453842168582491